<a href="https://colab.research.google.com/github/wangjuan001/sandbox2019/blob/master/Copy_of_tnseq_with_pandas2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Pandas : Part 2
------
This tutorial is heavily based on [Pandas in 10 min](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#). The original material waas modified by adding TnSeq data as examples.

In [0]:
import pandas as pd
import numpy as np
%matplotlib inline  

## Get datasets to play with

In [0]:
%%bash
wget https://nekrut.github.io/BMMB554/tnseq_untreated.txt.gz
wget https://nekrut.github.io/BMMB554/ta_gc.txt

In [0]:
data_file = 'tnseq_untreated.txt.gz'

In [0]:
# Just two choices for beginning of of gene field
!gunzip -c {data_file} | cut -f 8 | cut -f 1 -d '=' | sort | uniq -c

  49898 .
 220202 ID


In [0]:
# Process tnseq_untreated.txt.gz to correctly parse gene names

import os
f = open('data.txt','w')

with os.popen('gunzip -c {}'.format(data_file)) as stream:
  for line in stream:
    if line.split( '\t' )[7].startswith( '.' ):
      f.write( '{}\t{}\n'.format( '\t'.join( line.split( '\t' )[:7] ) , 'intergenic'  ) )
    elif line.split( '\t' )[7].startswith( 'ID' ):
      f.write( '{}\t{}\n'.format( '\t'.join( line.split( '\t' )[:7] ) , line.split( '\t' )[7].split(';')[0][3:] ) )
f.close()

In [0]:
# Read from the file

tnseq = pd.read_table('data.txt', header=None, names=['pos','blunt','cap','dual','erm','pen','tuf','gene'])

In [0]:
tnseq.head()

,pos,blunt,cap,dual,erm,pen,tuf,gene
0,2400002,0.0,0.0,1.0,0.0,0.0,1.0,intergenic
1,2400004,1.0,0.0,5.0,0.0,0.0,1.0,intergenic
2,2400006,1.0,0.0,5.0,1.0,0.0,1.0,intergenic
3,2400009,2.0,2.0,8.0,1.0,0.0,0.0,intergenic
4,2400029,6.0,1.0,0.0,1.0,0.0,1.0,intergenic


In [0]:
# Set position as index

tnseq = tnseq.set_index('pos')

In [0]:
tnseq.head()

,blunt,cap,dual,erm,pen,tuf,gene
pos,,,,,,,
2400002,0.0,0.0,1.0,0.0,0.0,1.0,intergenic
2400004,1.0,0.0,5.0,0.0,0.0,1.0,intergenic
2400006,1.0,0.0,5.0,1.0,0.0,1.0,intergenic
2400009,2.0,2.0,8.0,1.0,0.0,0.0,intergenic
2400029,6.0,1.0,0.0,1.0,0.0,1.0,intergenic


In [0]:
# Reading GC content data

gc = pd.read_table('ta_gc.txt', header=None, names=['pos','gc'])

In [0]:
gc.head()

,pos,gc
0,4,0.339286
1,10,0.354839
2,16,0.367647
3,42,0.372340
4,79,0.303922


In [0]:
# Set position as index as well

gc = gc.set_index('pos')

In [0]:
gc.head()

,gc
pos,
4,0.339286
10,0.354839
16,0.367647
42,0.372340
79,0.303922


## Joins of all sorts

![](http://kirillpavlov.com/images/join-types.png)

Image from Kirill Pavlov [blog](http://kirillpavlov.com/blog/2016/04/23/beyond-traditional-join-with-apache-spark/)

### Prepare sample data

To make things more digestable we will create twio dataframes, `df1` and `df2`, that are small subsets of `tnseq` and `gc` tables. In making them we will make sure that thay mostly overlap but also contain a few rows with indexes not present in the other dataframe.

In [0]:
# Let's create a small subset of tnseq data:

df1 = tnseq[( tnseq['gene'] != 'intergenic' ) & ( tnseq['blunt']>100 ) ].head(10)

In [0]:
# Create a numpy array contain index values from fd1

i = np.array(df1.index[1:])

In [0]:
i

array([2404933, 2404937, 2410079, 2410094, 2419997, 2430244, 2430254,
       2439462, 2439466])

In [0]:
# Append a few gc index values to i, that are not present in df1

i = np.append(i,[2410079,2405277,2405301])

In [0]:
i

array([2404933, 2404937, 2410079, 2410094, 2419997, 2430244, 2430254,
       2439462, 2439466, 2410079, 2405277, 2405301])

In [0]:
# ... and a subset of gc data

df2 = gc.loc[i]

In [0]:
# This is what we have in df1

df1

,blunt,cap,dual,erm,pen,tuf,gene
pos,,,,,,,
2404930,283.0,65.0,109.0,94.0,47.0,128.0,gene2465
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481
2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491
2430254,128.0,83.0,29.0,34.0,42.0,38.0,gene2491
2439462,112.0,35.0,36.0,93.0,22.0,56.0,gene2499


In [0]:
# ... and this is content of df2
df2

,gc
pos,
2404933,0.284314
2404937,0.274510
2410079,0.352941
2410094,0.333333
2419997,0.343137
2430244,0.274510
2430254,0.274510
2439462,0.313725
2439466,0.313725



### Inner join

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/SQL_Join_-_07_A_Inner_Join_B.svg/220px-SQL_Join_-_07_A_Inner_Join_B.svg.png)

Here **A** is `df1` and **B** is `df2`.

Image from [Wikipedia](https://en.wikipedia.org/wiki/Join_(SQL).

In [0]:
df1.join(df2, how = 'inner')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137
2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,0.274510
2430254,128.0,83.0,29.0,34.0,42.0,38.0,gene2491,0.274510
2439462,112.0,35.0,36.0,93.0,22.0,56.0,gene2499,0.313725


In [0]:
pd.merge(df1, df2, left_index=True, right_index=True, how = 'inner')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137
2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,0.274510
2430254,128.0,83.0,29.0,34.0,42.0,38.0,gene2491,0.274510
2439462,112.0,35.0,36.0,93.0,22.0,56.0,gene2499,0.313725


### Left join

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/SQL_Join_-_01_A_Left_Join_B.svg/220px-SQL_Join_-_01_A_Left_Join_B.svg.png)

Here **A** is `df1` and **B** is `df2`.

Image from [Wikipedia](https://en.wikipedia.org/wiki/Join_(SQL).

In [0]:
df1.join(df2, how = 'left')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404930,283.0,65.0,109.0,94.0,47.0,128.0,gene2465,NaN
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137
2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,0.274510
2430254,128.0,83.0,29.0,34.0,42.0,38.0,gene2491,0.274510


In [0]:
pd.merge(df1, df2, left_index=True, right_index=True, how = 'left')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404930,283.0,65.0,109.0,94.0,47.0,128.0,gene2465,NaN
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137
2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,0.274510
2430254,128.0,83.0,29.0,34.0,42.0,38.0,gene2491,0.274510


### Right join

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/SQL_Join_-_03_A_Right_Join_B.svg/220px-SQL_Join_-_03_A_Right_Join_B.svg.png)

Here **A** is `df1` and **B** is `df2`.

Image from [Wikipedia](https://en.wikipedia.org/wiki/Join_(SQL).

In [0]:
df1.join(df2, how = 'right')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2405277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.303922
2405301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137
2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,0.274510


In [0]:
pd.merge(df1, df2, left_index=True, right_index=True, how = 'right')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2405277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.303922
2405301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137
2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,0.274510


### Full join

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3d/SQL_Join_-_05b_A_Full_Join_B.svg/220px-SQL_Join_-_05b_A_Full_Join_B.svg.png)

Here **A** is `df1` and **B** is `df2`.

Image from [Wikipedia](https://en.wikipedia.org/wiki/Join_(SQL).

In [0]:
df1.join(df2, how = 'outer')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404930,283.0,65.0,109.0,94.0,47.0,128.0,gene2465,NaN
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2405277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.303922
2405301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137


In [0]:
pd.merge(df1, df2, left_index=True, right_index=True, how = 'outer')

,blunt,cap,dual,erm,pen,tuf,gene,gc
pos,,,,,,,,
2404930,283.0,65.0,109.0,94.0,47.0,128.0,gene2465,NaN
2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,0.284314
2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,0.274510
2405277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.303922
2405301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.274510
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,0.352941
2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,0.333333
2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,0.343137


## Grouping

-------

By “group by” we are referring to a process involving one or more of the following steps:

 - Splitting the data into groups based on some criteria
 - Applying a function to each group independently
 - Combining the results into a data structure
See the [Grouping section](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#groupby).

In [0]:
df1.groupby(['gene']).sum()

,blunt,cap,dual,erm,pen,tuf
gene,,,,,,
gene2465,920.0,211.0,332.0,310.0,182.0,429.0
gene2471,387.0,65.0,52.0,90.0,38.0,94.0
gene2481,111.0,37.0,35.0,50.0,39.0,66.0
gene2491,264.0,173.0,61.0,95.0,100.0,85.0
gene2499,220.0,65.0,68.0,149.0,41.0,101.0


In [0]:
df1.groupby(['gene']).max()

,blunt,cap,dual,erm,pen,tuf
gene,,,,,,
gene2465,353.0,79.0,115.0,122.0,80.0,173.0
gene2471,194.0,33.0,26.0,46.0,19.0,47.0
gene2481,111.0,37.0,35.0,50.0,39.0,66.0
gene2491,136.0,90.0,32.0,61.0,58.0,47.0
gene2499,112.0,35.0,36.0,93.0,22.0,56.0


## Actually using SQL

There is a great SQL-like interface for Pandas called [`pandasql`](https://github.com/yhat/pandasql):

In [0]:
!pip install -U pandasql

  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [0]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [0]:
## Aggregating

pysqldf("select gene, sum(blunt) as bl from df1 group by gene")

,gene,bl
0,gene2465,920.0
1,gene2471,387.0
2,gene2481,111.0
3,gene2491,264.0
4,gene2499,220.0


In [0]:
## Joining (left join)

pysqldf("select * from df1 left join df2 on df1.pos = df2.pos")

,pos,blunt,cap,dual,erm,pen,tuf,gene,pos,gc
0,2404930,283.0,65.0,109.0,94.0,47.0,128.0,gene2465,NaN,NaN
1,2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,2404933.0,0.284314
2,2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,2404937.0,0.274510
3,2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,2410079.0,0.352941
4,2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,2410079.0,0.352941
5,2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,2410094.0,0.333333
6,2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,2419997.0,0.343137
7,2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,2430244.0,0.274510
8,2430254,128.0,83.0,29.0,34.0,42.0,38.0,gene2491,2430254.0,0.274510
9,2439462,112.0,35.0,36.0,93.0,22.0,56.0,gene2499,2439462.0,0.313725


In [0]:
## Joining (inner join)

pysqldf("select * from df1 join df2 on df1.pos = df2.pos")

,pos,blunt,cap,dual,erm,pen,tuf,gene,pos,gc
0,2404933,284.0,67.0,108.0,94.0,55.0,128.0,gene2465,2404933,0.284314
1,2404937,353.0,79.0,115.0,122.0,80.0,173.0,gene2465,2404937,0.274510
2,2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,2410079,0.352941
3,2410094,194.0,33.0,26.0,46.0,19.0,47.0,gene2471,2410094,0.333333
4,2419997,111.0,37.0,35.0,50.0,39.0,66.0,gene2481,2419997,0.343137
5,2430244,136.0,90.0,32.0,61.0,58.0,47.0,gene2491,2430244,0.274510
6,2430254,128.0,83.0,29.0,34.0,42.0,38.0,gene2491,2430254,0.274510
7,2439462,112.0,35.0,36.0,93.0,22.0,56.0,gene2499,2439462,0.313725
8,2439466,108.0,30.0,32.0,56.0,19.0,45.0,gene2499,2439466,0.313725
9,2410079,193.0,32.0,26.0,44.0,19.0,47.0,gene2471,2410079,0.352941


PandaSQLException: ignored